In [1]:
import pandas as pd
import os
import zipfile
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime, Numeric, Boolean
from sqlalchemy.orm import sessionmaker, relationship, declarative_base
from datetime import datetime
from werkzeug.security import generate_password_hash

C:\Users\Alex\AppData\Local\Temp\ipykernel_21564\3449460678.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Создать соединение с базой данных в SQLAlchemy
# Замените 'sqlite:///mydatabase.db' на путь к вашей базе данных
engine = create_engine('sqlite:///./mydatabase.db')

In [3]:
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy import func


# Создаем базовый класс для объявления моделей
class Base(DeclarativeBase): pass

# Определение модели для таблицы пользователей
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True, autoincrement=True)
    username = Column(String(255), nullable=False)
    password_hash = Column(String(255), nullable=False)  # Предполагается, что это поле для хранения хэшированного пароля
    login = Column(String(255), nullable=False, unique=True)
    email = Column(String(255), nullable=False, unique=True)  # Добавлено поле email, как было запрошено ранее
    registration_date = Column(DateTime(timezone=True), default=func.now())

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)

# Определение модели для аккаунта
class Account(Base):
    __tablename__ = 'accounts'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    cash = Column(Integer, nullable=False)
    
    user = relationship('User', back_populates='accounts')

# Определение модели для таблицы действий
class Action(Base):
    __tablename__ = 'actions'

    id = Column(Integer, primary_key=True, autoincrement=True)
    account_id = Column(Integer, ForeignKey('accounts.id'))
    currency_spent = Column(Integer, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)

    user = relationship('User', back_populates='actions')

User.actions = relationship('Action', order_by=Action.id, back_populates='user')

# Определение модели для таблицы ML-моделей
class Model(Base):
    __tablename__ = 'models'

    id = Column(Integer, primary_key=True, autoincrement=True)
    model_name = Column(String, nullable=False)
    cost = Column(Integer, nullable=False)

# Определение модели для таблицы предсказаний
class Prediction(Base):
    __tablename__ = 'predictions'

    id = Column(Integer, primary_key=True, autoincrement=True)
    action_id = Column(Integer, ForeignKey("actions.id"))
    user_id = Column(Integer, ForeignKey('users.id'))
    model_id = Column(Integer, ForeignKey('models.id'))
    created_at = Column(DateTime, default=datetime.utcnow)

    gender = Column(Boolean, nullable=False, default=True)
    body_mass_index = Column(Float, nullable=False)
    physical_activity = Column(Boolean, nullable=False, default=True)
    insulin_level = Column(Float, nullable=False)
    diabetes = Column(Integer, nullable=True)
    glucose_level = Column(Float, nullable=False)
    glucose_tolerance_test = Column(Float, nullable=False)
    prediction_result = Column(String(255), nullable=False)

    action = relationship('Action', back_populates='predictions')
    user = relationship('User', back_populates='predictions')
    model = relationship('Model', back_populates='predictions')

Action.predictions = relationship('Prediction', order_by=Prediction.id, back_populates='action')
User.predictions = relationship('Prediction', order_by=Prediction.id, back_populates='user')
Model.predictions = relationship('Prediction', order_by=Prediction.id, back_populates='model')

# Создаем таблицы в базе данных, если они еще не существуют
Base.metadata.create_all(engine)

# Создаем сессию для работы с базой данных
Session = sessionmaker()
with Session(autoflush=False, bind=engine) as db: 
    pass

In [7]:
# Создание экземпляра пользователя
new_user = User(
    username='user123',
    email='123@example.com',
)
new_user.set_password('password123')

# Добавление участника в сессию
db.add(new_user)
db.commit()  # Сохранение изменений в базе данных
db.refresh(new_user) # Обновление состояния объекта

InvalidRequestError: Mapper 'Mapper[User(users)]' has no property 'accounts'.  If this property was indicated from other mappers or configure events, ensure registry.configure() has been called.

In [ ]:
# Извлечение всех участников
all_users = db.query(User).all()
for user in all_users:
    print(user.username, user.email, user.created_at)

In [ ]:
first_user = db.query(User).filter(User.id==1).first()
first_user.username = 'frog'
db.commit()
all_users = db.query(User).all()
for u in all_users:
    print(u.username, u.email)

In [ ]:
all_users = db.query(User).filter(User.id==1).first()
all_users.username = 'frog'
db.commit()
frog = db.query(User).all()
print(u.username, u.email)

In [ ]:
db.rollback()